In [1]:
import pandas as pd
import pymongo
from pymongo import MongoClient
import graphlab as gl
import csv
gl.canvas.set_target('ipynb')

In [3]:
client = MongoClient()
kiva = client.kiva
mongo_loans = kiva.loans
mongo_loans_lenders = kiva.loans_lenders

##Feature Engineering
This iteration will take into account several simple features: **activity, loan_amount, country, posted_date, sector**.

In [60]:
# get loans_lenders
cursor_loans_lenders = mongo_loans_lenders.find({}, {'_id':0}).sort('id', 1).limit(5000)
sample_loans_lenders = pd.DataFrame(list(cursor_loans_lenders))
sample_loans_lenders.dropna(inplace=True)

In [61]:
sample_loans_lenders.head()

,id,lender_ids
0,280242,"[sebastian3247, NkInteractClub, ivancho, greg7..."
1,280741,"[frank6713, zx81, brendan3108, christine5292, ..."
2,280758,"[katherine9284, susan10856289, geoff1088, mark..."
3,280766,"[christine7920, barbara4752, jasongerlach, dor..."
4,280776,"[kylie8236, kenneth2086, maria3264, raymond831..."


In [62]:
# create loan-lender pairs
with open('data/loans_lenders_5000.csv', 'w') as f:
    wr = csv.writer(f, delimiter=',')
    for r in sample_loans_lenders.iterrows():
        for l in r[1]['lender_ids']:
            wr.writerow([r[1]['id'], l])

In [63]:
sf = gl.SFrame.read_csv('data/loans_lenders_5000.csv', header=False, delimiter=',', verbose=True)
sf.rename({'X1':'loan_id', 'X2':'lender_id'}).show()

PROGRESS: Finished parsing file /Users/yulizhou/Documents/Projects/zipfian/Kipfian/data/loans_lenders_5000.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.116632 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/yulizhou/Documents/Projects/zipfian/Kipfian/data/loans_lenders_5000.csv
PROGRESS: Parsing completed. Parsed 127296 lines in 0.097173 secs.


<IPython.core.display.Javascript object>

In [88]:
# start getting features
df = sample_loans_lenders.drop(['lender_ids'], axis=1)
ids = list(df['id'].values)
c=mongo_loans.find({'id': {'$in': ids}}, 
                   {'id': 1, 'activity': 1, 'loan_amount': 1, 'sector': 1, 'location.country':1, 'posted_date': 1, '_id':0})\
                    .sort('id', 1)

In [ ]:
print len(list(c))
print len(ids)
# df = pd.concat([df, pd.Series(list(c))], ignore_index=False)